In [71]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from PIL import ImageEnhance, ImageFilter
from torchmetrics import Accuracy
from torchinfo import summary

# pretrained models (AlexNet) from torchvision

import torchvision.models as models

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
sys.path.append('../')  

from Models.alexnet import AlexNet

import numpy as np

In [72]:
base_path = os.getcwd() + '\\data'

In [109]:
class EdgeEnhancement:
    def __call__(self, img):
        return img.filter(ImageFilter.FIND_EDGES)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize(256),  # Resize slightly larger than final size
    transforms.RandomResizedCrop(224),  # Random crop back down to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally
    transforms.RandomRotation(15),  # Rotate by +/- 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, ),  # Randomly change brightness and contrast
    # transforms.Resize(224),  # Resize to 224x224 to match AlexNet input size
    # transforms.Lambda(lambda img: EdgeEnhancement()(img)),
    transforms.ToTensor(),   # Convert the image to a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])




class DropletDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
        Args:
            data_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Load images and labels
        for label in ['background', 'droplets']:
            class_dir = os.path.join(data_dir, label)
            for filename in os.listdir(class_dir):
                if filename.endswith('.jpg'):  # Modify if needed for different extensions
                    img_path = os.path.join(class_dir, filename)
                    self.images.append(img_path)
                    self.labels.append(1 if label == 'droplets' else 0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')  # Convert to RGB if not already
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        label = torch.tensor(label)
        return image, label
    

droplet_dataset = DropletDataset(data_dir=base_path, transform=transform)

dataloader = DataLoader(droplet_dataset, batch_size=32, shuffle=True, num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [110]:
# get alexnet pretrained on ImageNet
alexnet_droplet_v2 = models.alexnet(weights=True)


alexnet_droplet = AlexNet(num_classes=10, channels=1).to(device)

alexnet_droplet.load_state_dict(torch.load('alexnet_model_mnist_full.pth'))

<All keys matched successfully>

In [ ]:
summary(alexnet_droplet, input_size=(1, 1, 224, 224))

In [ ]:
summary(alexnet_droplet_v2, input_size=(1, 3, 224, 224))

In [116]:
# Modify the last layer of the classifier to output 2 classes instead of 10
alexnet_droplet.classifier[6] = nn.Linear(4096, 2).to(device)

alexnet_droplet_v2.classifier[6] = torch.nn.Linear(alexnet_droplet_v2.classifier[6].in_features, 2)

# image is also with 1 channel
alexnet_droplet_v2.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
alexnet_droplet_v2 = alexnet_droplet_v2.to(device)


# accuracy = Accuracy(task='multiclass', num_classes=2)

accuracy = Accuracy(task='multiclass', num_classes=2)

accuracy = accuracy.to(device)

In [ ]:
summary(alexnet_droplet_v2, input_size=(1, 1, 224, 224))

In [112]:
class_weights = torch.tensor([2.0, 1.0])  # Adjust these values based on your understanding of class importance
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

optimizer_adam = optim.Adam(alexnet_droplet.parameters(), lr=1e-1)
optimizer_sgd = torch.optim.SGD(alexnet_droplet.parameters(), lr=0.001, momentum=0.9)


# optim adam for alexnet_v2
optimizer_adam_v2 = optim.Adam(alexnet_droplet_v2.parameters(), lr=1e-4)
optimizer_sgd_v2 = torch.optim.SGD(alexnet_droplet_v2.parameters(), lr=0.001, momentum=0.9)

In [113]:
# Scheduler for learning rate decay
scheduler_adam = StepLR(optimizer_adam, step_size=10, gamma=0.1)


scheduler_adam_v2 = StepLR(optimizer_adam_v2, step_size=7, gamma=0.1)

In [175]:
# torch.Size([1, 224, 224])

# AlexNet on MNIST10

In [114]:
num_epochs = 40

for epoch in range(num_epochs):
    alexnet_droplet.train()
    train_loss, train_acc = 0, 0

    for images, labels in dataloader:

        images, labels = images.to(device), labels.to(device)

        optimizer_adam.zero_grad() # Zero the gradients

        outputs = alexnet_droplet(images) # Forward pass

        loss = criterion(outputs, labels) # Calculate the loss
        accuracy.update(outputs, labels)

        loss.backward() # Backward pass

        optimizer_adam.step() # Update the weights

        train_loss += loss.item()

    scheduler_adam.step() # Step the scheduler

    train_loss /= len(droplet_dataset)
    train_acc = accuracy.compute()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

Epoch 1/40, Loss: 305747.1719, Accuracy: 0.5600
Epoch 2/40, Loss: 1486398.4133, Accuracy: 0.5267
Epoch 3/40, Loss: 66583.0197, Accuracy: 0.5222
Epoch 4/40, Loss: 64859.9824, Accuracy: 0.5033
Epoch 5/40, Loss: 10.0272, Accuracy: 0.4880
Epoch 6/40, Loss: 91058.9718, Accuracy: 0.4967
Epoch 7/40, Loss: 1597.8118, Accuracy: 0.4924
Epoch 8/40, Loss: 139.8350, Accuracy: 0.5133
Epoch 9/40, Loss: 245712.2717, Accuracy: 0.5007
Epoch 10/40, Loss: 102.0761, Accuracy: 0.4940
Epoch 11/40, Loss: 0.2803, Accuracy: 0.5018
Epoch 12/40, Loss: 0.2913, Accuracy: 0.4978
Epoch 13/40, Loss: 0.2723, Accuracy: 0.4933
Epoch 14/40, Loss: 0.2205, Accuracy: 0.4933
Epoch 15/40, Loss: 0.1823, Accuracy: 0.4920
Epoch 16/40, Loss: 0.2116, Accuracy: 0.4900
Epoch 17/40, Loss: 0.1824, Accuracy: 0.4906
Epoch 18/40, Loss: 0.1917, Accuracy: 0.4852
Epoch 19/40, Loss: 0.1235, Accuracy: 0.4895
Epoch 20/40, Loss: 0.1554, Accuracy: 0.4857
Epoch 21/40, Loss: 0.0908, Accuracy: 0.4917
Epoch 22/40, Loss: 0.1364, Accuracy: 0.4942
Epoch

# AlexNet on ImageNet

In [117]:
num_epochs = 40

for epoch in range(num_epochs):
    alexnet_droplet_v2.train()
    train_loss, train_acc = 0, 0

    for images, labels in dataloader:

        images, labels = images.to(device), labels.to(device)

        optimizer_adam_v2.zero_grad() # Zero the gradients

        outputs = alexnet_droplet_v2(images) # Forward pass

        loss = criterion(outputs, labels) # Calculate the loss
        accuracy.update(outputs, labels)

        loss.backward() # Backward pass

        optimizer_adam_v2.step() # Update the weights

        train_loss += loss.item()

    scheduler_adam.step() # Step the scheduler

    train_loss /= len(droplet_dataset)
    train_acc = accuracy.compute()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

Epoch 1/40, Loss: 0.0246, Accuracy: 0.5000
Epoch 2/40, Loss: 0.0236, Accuracy: 0.5500
Epoch 3/40, Loss: 0.0228, Accuracy: 0.5444
Epoch 4/40, Loss: 0.0222, Accuracy: 0.5783
Epoch 5/40, Loss: 0.0210, Accuracy: 0.6040
Epoch 6/40, Loss: 0.0212, Accuracy: 0.6000
Epoch 7/40, Loss: 0.0217, Accuracy: 0.6076
Epoch 8/40, Loss: 0.0217, Accuracy: 0.6075
Epoch 9/40, Loss: 0.0214, Accuracy: 0.6007
Epoch 10/40, Loss: 0.0214, Accuracy: 0.6067
Epoch 11/40, Loss: 0.0195, Accuracy: 0.6139
Epoch 12/40, Loss: 0.0180, Accuracy: 0.6233
Epoch 13/40, Loss: 0.0208, Accuracy: 0.6297
Epoch 14/40, Loss: 0.0187, Accuracy: 0.6310
Epoch 15/40, Loss: 0.0180, Accuracy: 0.6396
Epoch 16/40, Loss: 0.0167, Accuracy: 0.6450
Epoch 17/40, Loss: 0.0155, Accuracy: 0.6533
Epoch 18/40, Loss: 0.0175, Accuracy: 0.6559
Epoch 19/40, Loss: 0.0166, Accuracy: 0.6625
Epoch 20/40, Loss: 0.0146, Accuracy: 0.6670
Epoch 21/40, Loss: 0.0150, Accuracy: 0.6721
Epoch 22/40, Loss: 0.0132, Accuracy: 0.6791
Epoch 23/40, Loss: 0.0121, Accuracy: 0.68

In [66]:
# Create a test scenario
accuracy_ = Accuracy(task='multiclass', num_classes=2, average='macro')
predictions = torch.tensor([[2.0, 1.0], [1.0, 3.0], [0.5, 0.2]])
labels = torch.tensor([0, 1, 0])

# Calculate accuracy
print(accuracy_(predictions, labels))

tensor(1.)
